<a href="https://colab.research.google.com/github/shivanishimpi/blogs/blob/main/kerasTuner_intro_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> This notebook has all the code used in the [keras-tuner blog](https://shivanishimpi9.medium.com/choosing-the-right-set-of-hyperparameters-keras-tuner-introduction-implementation-d4c387f4d655?postPublishedType=repub) 


Date Created: January 12, 2021

**Author:** Shivani Shimpi

Drop me an [email](mailto:shivanishimpi9@gmail.com) or reach out to me on -
[[GitHub](https://github.com/shivanishimpi)] [[LinkedIn](https://www.linkedin.com/in/shivani-shimpi-5113a8170/)]

Note: 

This tutorial is not going to help you to build TensorFlow models, instead will be a guide to using `keras-tuner` on your existing models. Therefore these are certain code blocks that will help you modify and train your models efficiently.

[Click here](https://github.com/shivanishimpi/blogs/blob/main/ANN_with_keras_tuner.ipynb) to get the executable code.

[Click here](https://shivanishimpi9.medium.com/choosing-the-right-set-of-hyperparameters-keras-tuner-introduction-implementation-d4c387f4d655) for the medium blog.

## classical-model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation

def build_model():
  model = keras.models.Sequential()
  model.add(Dense(16))
  model.add(Activation('relu'))
  model.add(Dense(32))
  model.add(Activation('relu'))
  model.add(Dense(1))
  
  model.compile(
    loss='mean_absolute_error',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics = ['mae','mse'])
  
  return model

model = build_model()

numEpochs = 50

history = model.fit(
  trainFeatures,
  trainLabels,
  epochs=numEpochs,
  validation_data=(testFeatures,testLabels)
)

##keras-tuner

In [ ]:
!pip install -U keras-tuner

In [ ]:
import kerastuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time

In [ ]:
LOG_DIR = f"{int(time.time())}"

def build_model(hp):
  model = keras.models.Sequential()
  model.add(Dense(
    hp.Int('input_units', min_value=12, max_value=256),
    input_shape=trainFeatures.shape[1:]
  ))
  model.add(Activation('relu'))
  
  for i in range(hp.Int("n_layers", min_value=1, max_value=5)):
    model.add(Dense(hp.Int(f"dense_{i}_units", 32, 256)))
    model.add(Activation('relu'))
  
  model.add(Dense(1))
  
  model.compile(
    loss='mean_absolute_error',
    optimizer=tf.keras.optimizers.Adam(
      hp.Choice('lr',[0.001,0.01,0.002,0.2])
    ),
    metrics = ['mae','mse'])
    
  return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials = 10,
    executions_per_trial = 2,
    directory = LOG_DIR
)

In [ ]:
tuner.search(
    x = trainFeatures,
    y = trainLabels,
    epochs = 50,
    validation_data = (testFeatures ,testLabels)
)

In [ ]:
tuner.results_summary() #returns top ten results

In [ ]:
tuner.get_best_models()[0].summary() #returns the best model

In [ ]:
#To save the model for resue

import pickle
#Saves the tuner as pickle obje
with open(f'tuner_{int(time.time())}.pkl', 'wb') as f:
    pickle.dump(tuner, f)
#Loads the former object
tuner = pickle.load(open('tuner_1610218332.pkl','rb'))

## Conclusions



That’s it from my side. Thanks for reading! Hope you learned something new.
Reach out to me if you have any suggestions or questions. It would encourage me to continue writing, besides, I will be more than happy to receive them.

You can check out my [website](http://shivanishimpi.github.io/) for more updates. Follow me on [Medium](https://shivanishimpi9.medium.com/) and [GitHub](github.com/shivanishimpi) for more such content. Let’s also connect on [LinkedIn](https://www.linkedin.com/in/shivani-shimpi-5113a8170/).


To get a video illustration of this you can check out the [YouTube video](https://youtu.be/DQuS_cZg2wI?t=3648)